In the feature pipeline we donwload the relevant dataset from HuggingFace and save it to Google Drive for later use. This includes processing the data using a processer which turns the raw audio data into Log-Mel Spectrogram features and tokenises the related text. The processor is also saved on google drive for convenience.

We begin by importing the dependencies

In [1]:
from huggingface_hub import notebook_login
import os
from datasets import load_dataset, DatasetDict
from transformers import WhisperProcessor
import hopsworks
from datasets import Audio

Login to the HF hub

In [3]:
notebook_login()

Download the Common Voice dataset and process it before it is stored

In [ ]:
common_voice = DatasetDict()

common_voice["train"] = load_dataset(
    "mozilla-foundation/common_voice_13_0", "sv-SE", split="train+validation"
)
common_voice["test"] = load_dataset(
    "mozilla-foundation/common_voice_13_0", "sv-SE", split="test"
)

common_voice = common_voice.select_columns(["audio", "sentence"])

print(common_voice)
print(common_voice["train"].features)

Process the dataset

In [ ]:
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language="Swedish", task="transcribe"
)

sampling_rate = processor.feature_extractor.sampling_rate
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=sampling_rate))

def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["sentence"],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

common_voice = common_voice.map(
    prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1
)

max_input_length = 30.0


def is_audio_in_length_range(length):
    return length < max_input_length

common_voice["train"] = common_voice["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

print(common_voice["train"])
print(common_voice["train"][0])

Save processor and dataset to GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
processor.save_pretrained("/content/gdrive/MyDrive/Models/whisper-small-Swedish")
common_voice.save_to_disk("/content/gdrive/MyDrive/common_voice_features")